<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/eceg_koblitz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elliptic Curve Parameter

In [ ]:
# Parameters
p = 751  # prime modulus
k = 20    # chosen small integer
a = -1    # coefficient of x in the elliptic curve
b = 188    # constant in the elliptic curve

# Koblitz Method

## Char to Num
represent 1 - 9 and a = 10 ..... z = 35

In [ ]:
# Manually create dictionaries for char-to-num and num-to-char mappings
char_to_num_dict = {
    '0': 10, '1': 11,
    'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19,
    'i': 20, 'j': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27,
    'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'w': 34, 'x': 35
}

# Reverse dictionary for num-to-char mapping
num_to_char_dict = {v: k for k, v in char_to_num_dict.items()}

# Function to map char to num
def char_to_num(char):
    if char in char_to_num_dict:
        return char_to_num_dict[char]
    else:
        raise ValueError(f"Character '{char}' is not valid. Please use '0-1' or 'a-x'.")

# Function to map num to char
def num_to_char(num):
    if num in num_to_char_dict:
        return num_to_char_dict[num]
    else:
        raise ValueError(f"Number '{num}' is not valid. Please use numbers in the range 10-35.")

def message_to_koblitz(message):
    # Convert the message to a list of characters
    chars = list(message)

    koblitz = []
    for char in chars:
        num = char_to_num(char)
        koblitz.append(num)

    return koblitz

def koblitz_to_message(koblitz):
    message = []
    for num in koblitz:
        char = num_to_char(num)
        message.append(char)

    return ''.join(message)

# Test mapping from char to num
chars = '01ax'

koblitz_message = message_to_koblitz(chars)
print("Koblitz Message:", koblitz_message)

message = koblitz_to_message(koblitz_message)
print("Message:", message)

Koblitz Message: [10, 11, 12, 35]
Message: 01ax


## Koblitz Encoding

In [ ]:
import sympy as sp

# Elliptic curve equation: y^2 = x^3 + ax + b mod p
def koblitz_encode(m, max_attempts=1000):
    num = 1  # Start with x = m * k + 1
    attempts = 0

    while attempts < max_attempts:
        x = m * k + num
        rhs = (x**3 + a * x + b) % p  # right-hand side of the elliptic curve equation

        # Check if rhs is a quadratic residue modulo p
        if sp.is_quad_residue(rhs, p):
            y = sp.sqrt_mod(rhs, p)
            return (x, y)  # Return the point (x, y) as a tuple

        num += 1  # Increment to check next x value
        attempts += 1

    # If no valid point is found after max_attempts
    raise ValueError(f"No valid point found after {max_attempts} attempts for message {m}.")

def koblitz_encode_message(message):
  encoded_points = []
  for char in message:
    encoded_point = koblitz_encode(char)
    encoded_points.append(encoded_point)
    print(f"Encoded point for character '{char}': {encoded_point}")

  return encoded_points

encoded_points = koblitz_encode_message(koblitz_message)
encoded_points

Encoded point for character '10': (201, 5)
Encoded point for character '11': (224, 248)
Encoded point for character '12': (241, 230)
Encoded point for character '35': (701, 203)


[(201, 5), (224, 248), (241, 230), (701, 203)]

## Koblitz Decoding

In [ ]:
# Decoding: m = (x - 1) / k
def koblitz_decode(x):
    return (x - 1) // k

def koblitz_decode_message(points):
    decoded_message = []
    for point in points:
        decoded_num = koblitz_decode(point[0])  # Decode the x-coordinate
        decoded_message.append(decoded_num)
        print(f"Decoded character for point {point}: {decoded_num}")
    return decoded_message

# Output the result
decoded_points = koblitz_decode_message(encoded_points)
decoded_points

Decoded character for point (201, 5): 10
Decoded character for point (224, 248): 11
Decoded character for point (241, 230): 12
Decoded character for point (701, 203): 35


[10, 11, 12, 35]

## Represented Integer to Char

In [ ]:
message = koblitz_to_message(decoded_points)
message

'01ax'

# Elliptic Curve El Gamal Class

In [ ]:
# Parameters
p = 11  # prime modulus
# k = 20    # chosen small integer
a = 1    # coefficient of x in the elliptic curve
b = 6    # constant in the elliptic curve
# B = (2, 4)

In [ ]:
# Parameters
p = 751  # prime modulus
a = -1    # coefficient of x in the elliptic curve
b = 188    # constant in the elliptic curve

In [ ]:
import random
import sympy as sp
import json
import base64

class EllipticCurveElGamal:
  def __init__(self, a, b, p, k, B=None):
    self.p = p # primer number
    self.a = a # alpha
    self.b = b # beta

    self.k = k
    self.PointB = B

    self.pointP = None
    self.pointQ = None


    self.char_to_num_dict = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
    'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17,
    'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25,
    'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33,
    'y': 34, 'z': 35, '.': 36, '#': 37, '*': 38, '/': 39, '-': 40
    }

    self.num_to_char_dict = {v: k for k, v in self.char_to_num_dict.items()}

    self.k_koblitz = max(self.char_to_num_dict.values()) + 1  # max 24 for 35 char

  def example_function(self):
      print(f"Using persistent k value: {self.k}")

  def elliptic_curve_equation(self, x):
    return (x**3 + self.a*x + self.b) % self.p

  def is_on_curve(self, x, y):
    return self.elliptic_curve_equation(x)  == (y**2) % p

  # def generate_random_number(self):
  #   self.k = random.randint(1, self.p - 1)
  #   return self.k

  def generate_random_point(self):
    while True:
      x = random.randint(1, self.p - 1)
      y = random.randint(1, self.p - 1)
      if self.is_on_curve(x, y):
        return (x, y)

  def mod_inverse(self, a, p):
      if a == 0:
          raise ValueError("Inverse does not exist for 0.")
      return pow(a, p - 2, p)

  def calculate_slope_mod(self, P, Q):
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 == y2:
        # Use the formula for point doubling when P == Q
        numerator = (3 * x1**2 + self.a) % self.p
        denominator = (2 * y1) % self.p
    else:
        # Use the formula for regular slope when P != Q
        numerator = (y2 - y1) % self.p
        denominator = (x2 - x1) % self.p

    if denominator == 0:
        raise ValueError("Slope is undefined (denominator is zero).")

    # Compute the slope as (numerator / denominator) % p, which is
    # numerator * mod_inverse(denominator, p) % p
    slope = (numerator * self.mod_inverse(denominator, self.p)) % self.p
    return slope

  def calculate_add_xr_mod(self, P, Q, m):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - x1 - x2) % self.p
    return xr

  def calculate_add_yr_mod(self, P, Q, m, xr):
    x1, y1 = P
    x2, y2 = Q
    yr = (m * (x1 - xr) - y1) % self.p
    return yr

  def calculate_point_addition(self, P, Q):
    m = self.calculate_slope_mod(P, Q)

    xr = self.calculate_add_xr_mod(P, Q, m)

    yr = self.calculate_add_yr_mod(P, Q, m, xr)

    R = (xr, yr)

    return R

  def calculate_dob_xr_mod(self, P, Q, m):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - (2 * x1)) % self.p
    return xr

  def calculate_dob_yr_mod(self, P, Q, m, xr):
    x1, y1 = P
    yr = (m * (x1 - xr) - y1) % self.p
    return yr

  def calculate_point_doubling(self, P):

    m = self.calculate_slope_mod(P, P)

    xr = self.calculate_dob_xr_mod(P, P, m)

    yr = self.calculate_dob_yr_mod(P, P, m, xr)

    R = (xr, yr)

    return R

  def calculate_point_multiplication(self, P, k):
    if k == 0:
      return None
    elif k == 1:
      return P

    # Initialize R to be the point at infinity, often represented as None in this context
    R = None
    Q = P  # Start with Q as P

    # Double-and-add method
    while k > 0:
        if k % 2 == 1:  # If k is odd, add Q to the result
            if R is None:
                R = Q  # R is the point at infinity initially
            else:
                R = self.calculate_point_addition(R, Q)
        Q = self.calculate_point_addition(Q, Q)  # Double the point Q
        k //= 2  # Move to the next bit

    return R

  def calculate_point_subtract(self, P, Q):
    # Find the inverse of point Q (x_Q, y_Q) -> (x_Q, -y_Q mod p)
    Q_inv = (Q[0], (-Q[1]) % self.p)

    # Subtract P - Q by adding P and Q_inv
    R = self.calculate_point_addition(P, Q_inv)

    return R

  def generate_public_key(self, B, private_key):
    public_key = self.calculate_point_multiplication(B, private_key)
    return public_key

  def enryption(self, M, public_key):
    # k = random.randint(1, self.p - 1) # 1 < k < p - 1

    C1 = self.calculate_point_multiplication(self.PointB, self.k)
    C2 = self.calculate_point_addition(M, self.calculate_point_multiplication(public_key, self.k))

    return (C1, C2)

  def decryption(self, C, private_key):
    C1, C2 = C

    _p = self.calculate_point_multiplication(C1, private_key)

    plain = self.calculate_point_subtract(C2, _p)

    return plain

  # Function to map char to num
  def char_to_num(self, char):
      # Ensure the input is a string; if not, convert it
      char = str(char)

      # Check if the string exists in the dictionary
      if char in self.char_to_num_dict:
          return self.char_to_num_dict[char]
      else:
          # Handle the expanded character set (0-40)
          raise ValueError(f"Character '{char}' is not valid. Please use '0-9', 'a-z', or other valid characters.")


  # Function to map num to char
  def num_to_char(self, num):
      # Ensure that the input is an integer; if it's a string digit, convert to integer
      if isinstance(num, str) and num.isdigit():
          num = int(num)

      # Now we expect num to be an integer
      if isinstance(num, int):
          if num in self.num_to_char_dict:  # Handle numbers 0-40 by converting them using the dictionary
              return self.num_to_char_dict[num]
          else:
              raise ValueError(f"Number '{num}' is out of the valid range (0-40).")
      else:
          raise ValueError(f"Input '{num}' is not valid. Please provide a valid number.")


  def message_to_koblitz(self, message):
      # Convert the message to a list of characters
      chars = list(message)

      koblitz = []
      for char in chars:
          num = self.char_to_num(char)
          koblitz.append(num)

      return koblitz

  def koblitz_to_message(self, koblitz):
      message = []
      for num in koblitz:
          char = self.num_to_char(num)
          message.append(char)

      return ''.join(message)

  def koblitz_encode(self, m, max_attempts=1000):
        m = m % self.k_koblitz  # Ensure the number is always within the valid range [0, k_koblitz)
        num = 1  # Start with x = m * k_koblitz + 1
        attempts = 0

        while attempts < max_attempts:
            x = m * self.k_koblitz + num
            rhs = (x ** 3 + self.a * x + self.b) % self.p  # Right-hand side of the elliptic curve equation

            # Check if rhs is a quadratic residue modulo p
            if sp.is_quad_residue(rhs, self.p):
                y = sp.sqrt_mod(rhs, self.p)
                return (x, y)  # Return the point (x, y) as a tuple

            num += 1  # Increment to check the next x value
            attempts += 1

        # If no valid point is found after max_attempts
        raise ValueError(f"No valid point found after {max_attempts} attempts for message {m}.")

  def koblitz_encode_message(self, message):
    encoded_points = []
    for char in message:
      encoded_point = self.koblitz_encode(char)
      encoded_points.append(encoded_point)
      # print(f"Encoded point for character '{char}': {encoded_point}")

    return encoded_points

  def koblitz_decode(self, x):
        m = (x - 1) // self.k_koblitz
        m = m % self.k_koblitz  # Ensure decoded number is within the valid range
        return m

  def koblitz_decode_message(self, points):
      decoded_message = []
      for point in points:
          decoded_num = self.koblitz_decode(point[0])  # Decode the x-coordinate
          decoded_message.append(decoded_num)
          # print(f"Decoded character for point {point}: {decoded_num}")
      return decoded_message

  def split_number(self, number):
        """Split a large number into chunks within the range 0-40"""
        chunks = []
        while number > 0:
            chunks.append(number % self.k_koblitz)  # Split off the last digit in base k_koblitz (41)
            number //= self.k_koblitz  # Shift to the next chunk
        return chunks[::-1]  # Return chunks in reverse order (most significant first)

  def combine_chunks(self, chunks):
        """Combine chunks back into the original number"""
        number = 0
        for chunk in chunks:
            number = number * self.k_koblitz + chunk  # Combine chunks back using base k_koblitz (41)
        return number

  def encrypt_message(self, message, public_key):
        # Step 1: Convert the message into Koblitz points
        koblitz_message = self.message_to_koblitz(message)
        encoded_message = self.koblitz_encode_message(koblitz_message)

        encrypted_points = []

        # Step 2: Encrypt each Koblitz point using elliptic curve encryption
        for point in encoded_message:
            encrypted_point = self.enryption(point, public_key)
            print(encrypted_point)
            # Use only the x-coordinate of C2 for encryption (C2 is the second point in the encryption tuple)
            encrypted_x_coord = encrypted_point[1][0]  # C2's x-coordinate

            # Step 3: Split the x-coordinate into smaller chunks (if it's a large number)
            split_chunks = self.split_number(encrypted_x_coord)

            # Step 4: Convert each chunk to characters and append to encrypted_points
            encrypted_points.extend([self.num_to_char(chunk) for chunk in split_chunks])
        # Step 5: Join all encrypted characters into a single string and return it as ciphertext
        return ''.join(encrypted_points)

  def decrypt_message(self, encrypted_message, private_key):
    # Step 1: Convert the encrypted message (characters) back to individual numbers
    encrypted_numbers = [self.char_to_num(char) for char in encrypted_message]
    print(encrypted_numbers)

    # Step 2: Group numbers into chunks and combine them back into x-coordinates
    combined_numbers = []  # This will store combined x-coordinates
    for i in range(0, len(encrypted_numbers), 2):  # Assuming 2 chunks per x-coordinate
        combined_number = self.combine_chunks(encrypted_numbers[i:i+2])  # Combine chunks back into the number
        combined_numbers.append(combined_number)

    print(combined_numbers)

    points = []
    for num in combined_numbers:
        try:
            point = self.koblitz_encode(num)  # Recover the original point using Koblitz encoding
            points.append(point)
        except ValueError as e:
            print(f"Error decoding point: {e}")

    decrypted_points = []

    # Step 3: Decrypt each Koblitz point using elliptic curve decryption
    for point in points:
        decrypted_point = self.decryption(point, private_key)
        decrypted_points.append(decrypted_point)

    # Step 4: Decode the decrypted Koblitz points into the original numeric values
    decoded_koblitz_points = self.koblitz_decode_message(decrypted_points)

    # Step 5: Convert the numeric Koblitz points back to characters and return the original message
    return self.koblitz_to_message(decoded_koblitz_points)


In [ ]:
random_num = random.randint(1, p - 1)
print(f'random number: {random_num}')

elliptic1 = EllipticCurveElGamal(a, b, p, random_num)

# k = elliptic1.generate_random_number()
# print(f'k: {k}')

basis = elliptic1.generate_random_point()
print(f'basis: {basis}')

elliptic1.PointB = basis

print(f'is basis on curve: {elliptic1.is_on_curve(basis[0], basis[1])}')

random number: 629
basis: (367, 122)
is basis on curve: True


In [ ]:
private_key = random.randint(1, p - 1)
public_key = elliptic1.generate_public_key(basis, private_key)

print(f'private key: {private_key}')
print(f'public key: {public_key}')

private key: 498
public key: (24, 686)


In [ ]:
message = (385, 423)

cipher = elliptic1.enryption(message, public_key)

decrypted_message = elliptic1.decryption(cipher, private_key)

print(f'message: {message}')
print(f'cipher: {cipher}')
print(f'decrypted message: {decrypted_message}')

message: (385, 423)
cipher: ((486, 355), (635, 242))
decrypted message: (385, 423)


## Testing

In [ ]:
CONTENT = "3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup"

In [ ]:
elliptic1.k_koblitz = 18 # max 18 if add "-#./*"

# plaintexts = "-#./*"
plaintext = "lorem.#-/*"
# plaintexts = CONTENT
print(f'Plaintext: {plaintext}')

ciphertext = elliptic1.encrypt_message(plaintext, public_key)
print(f"Ciphertext: {ciphertext}")

# Decrypt the ciphertext back to the original message
decrypted_message = elliptic1.decrypt_message(ciphertext, private_key)
print(f"Decrypted message: {decrypted_message}")

Plaintext: lorem.#-/*
((486, 355), (1, 375))
((486, 355), (2, 378))
((486, 355), (47, 335))
((486, 355), (423, 135))
((486, 355), (196, 113))
((486, 355), (629, 206))
((486, 355), (364, 354))
((486, 355), (196, 113))
((486, 355), (1, 375))
((486, 355), (225, 634))
Ciphertext: 122b159ag1gh124ag1c9
[1, 2, 2, 11, 1, 5, 9, 10, 16, 1, 16, 17, 1, 2, 4, 10, 16, 1, 12, 9]
[20, 47, 23, 172, 289, 305, 20, 82, 289, 225]


TypeError: cannot unpack non-iterable int object

In [ ]:
ciphertext

In [ ]:
class EllipticCurve:
    def __init__(self, a, b, p, k, B=None):
        self.p = p  # Prime number
        self.a = a  # Alpha
        self.b = b  # Beta
        self.k = k
        self.PointB = B

        self.char_to_num_dict = {
            '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
            'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17,
            'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25,
            'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33,
            'y': 34, 'z': 35, '.': 36, '#': 37, '*': 38, '/': 39, '-': 40
        }

        self.num_to_char_dict = {v: k for k, v in self.char_to_num_dict.items()}

        # Dynamically set k_koblitz based on the size of the character set
        self.k_koblitz = max(self.char_to_num_dict.values()) + 1  # This will be 41.

    def split_number(self, number):
        """Split a large number into chunks within the range 0-40"""
        chunks = []
        while number > 0:
            chunks.append(number % self.k_koblitz)  # Split off the last digit in base k_koblitz (41)
            number //= self.k_koblitz  # Shift to the next chunk
        return chunks[::-1]  # Return chunks in reverse order (most significant first)

    def combine_chunks(self, chunks):
        """Combine chunks back into the original number"""
        number = 0
        for chunk in chunks:
            number = number * self.k_koblitz + chunk  # Combine chunks back using base k_koblitz (41)
        return number

    def number_to_chars(self, number):
        """Convert a large number into characters"""
        chunks = self.split_number(number)  # Split the number into valid chunks
        chars = ''.join([self.num_to_char_dict[chunk] for chunk in chunks])  # Convert each chunk to a character
        return chars

    def chars_to_number(self, chars):
        """Convert characters back into the original number"""
        chunks = [self.char_to_num_dict[char] for char in chars]  # Convert each character back to a number
        number = self.combine_chunks(chunks)  # Combine the chunks back into the original number
        return number

    def encode_number_as_chars(self, number):
        """Encode the number as characters and print the result"""
        char_representation = self.number_to_chars(number)
        print(f"Character representation of {number}: {char_representation}")
        return char_representation

    def decode_chars_to_number(self, chars):
        """Decode characters back into the original number and print the result"""
        number_representation = self.chars_to_number(chars)
        print(f"Number representation of {chars}: {number_representation}")
        return number_representation

    def encode_and_decode(self, number):
        """Encode a number to chars and then decode it back to the number"""
        # Encode the number to characters
        encoded_chars = self.encode_number_as_chars(number)

        # Decode the characters back to the original number
        decoded_number = self.decode_chars_to_number(encoded_chars)

        return encoded_chars, decoded_number


In [ ]:
# Instantiate the class with elliptic curve parameters
elliptic_curve = EllipticCurve(a=2, b=3, p=17, k=24)

# Example number to test the encode > decode cycle
test_number = 621  # Example number to encode and decode

# Perform the process: Encode number -> Decode chars back to number
encoded_chars, decoded_number = elliptic_curve.encode_and_decode(test_number)
print(f"Final encoded chars: {encoded_chars}")
print(f"Final decoded number: {decoded_number}")

In [1]:
# Define the custom character set
char_set = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ*-&_##@:;"
base = len(char_set)  # Base is 72 because there are 72 characters in the set

# Convert a number to custom base-72 using the provided character set
def convert_to_custom_base(num, char_set):
    base = len(char_set)
    encoded_string = []
    while num > 0:
        remainder = num % base
        encoded_string.append(char_set[remainder])
        num //= base
    return ''.join(reversed(encoded_string))

# Convert a base-72 encoded string back to an integer
def convert_from_custom_base(encoded_string, char_set):
    base = len(char_set)
    decoded_num = 0
    for char in encoded_string:
        decoded_num = decoded_num * base + char_set.index(char)
    return decoded_num

# Chunk a string into chunks of size 2
def chunk_string(s, chunk_size=2):
    return [s[i:i+chunk_size] for i in range(0, len(s), chunk_size)]

# Join a list of 2-character chunks into a single string
def join_chunks(chunks):
    return ''.join(chunks)

# Encode two large integers x and y as a string of 2-character chunks
def encode_cipher(x, y, char_set):
    # Convert both x and y into base-72 encoded strings
    encoded_x = convert_to_custom_base(x, char_set)
    encoded_y = convert_to_custom_base(y, char_set)

    # Combine both encoded strings
    combined_encoded = encoded_x + encoded_y

    # Chunk the combined string into 2-character blocks
    cipher_chunks = chunk_string(combined_encoded, 2)

    # Join the chunks into a final ciphertext string
    return join_chunks(cipher_chunks)

# Decode the 2-character chunked ciphertext back into the original integers x and y
def decode_cipher(ciphertext, char_set):
    # Chunk the ciphertext into 2-character blocks
    chunks = chunk_string(ciphertext, 2)

    # Join the chunks into a single string
    encoded_string = join_chunks(chunks)

    # Let's assume we know where x and y were separated in the original encoding
    # For simplicity, let's split in the middle:
    mid_index = len(encoded_string) // 2
    encoded_x = encoded_string[:mid_index]
    encoded_y = encoded_string[mid_index:]

    # Convert the base-72 encoded strings back to integers
    x = convert_from_custom_base(encoded_x, char_set)
    y = convert_from_custom_base(encoded_y, char_set)

    return x, y

# Example usage without using a main block
# Example large integers for x and y (this would come from ElGamal encryption)
x = 1234567890987654321
y = 9876543210123456789

# Encoding
encoded_cipher = encode_cipher(x, y, char_set)
print("Encoded Ciphertext:", encoded_cipher)

# Decoding
decoded_x, decoded_y = decode_cipher(encoded_cipher, char_set)
print("Decoded x:", decoded_x)
print("Decoded y:", decoded_y)

Encoded Ciphertext: q_WF@DM@PL32tGQq38ZDe
Decoded x: 1234567890987654321
Decoded y: 9876543210123456789
